In [ ]:
### MDP Value Iteration and Policy Iteration
import argparse
import numpy as np
import gym
import time
#from lake_envs import *

np.set_printoptions(precision=3)

parser = argparse.ArgumentParser(description='A program to run assignment 1 implementations.',
								formatter_class=argparse.ArgumentDefaultsHelpFormatter)

parser.add_argument("--env", 
					help="The name of the environment to run your algorithm on.", 
					choices=["Deterministic-4x4-FrozenLake-v0","Stochastic-4x4-FrozenLake-v0"],
					default="Deterministic-4x4-FrozenLake-v0")


def policy_evaluation(P, nS, nA, policy, gamma=0.9, tol=1e-3):

  iter = 3000 #set maximum of iteration
  value_function = np.zeros(nS)
  
 
  i = 1
  d = tol+1
  while d > tol and i <= iter:
    value_function_new = np.zeros(nS)
    for s in range(nS):
      # for a in range(nA):
				# P[s][a][0] - transition probability
				# P[s][a][1] - next state
				# P[s][a][2] - reward
				# P[s][a][3] - 1 if next state is target 0 otherwise
      a = policy[s]
      for p, s_, r, t in P[s][a]:
        # if t: #if target                   
        #   value_function_new[s] += policy[s]*p*r #no next 
        # else:
        if not t:
          value_function_new[s] += r+gamma*p*value_function[int(s_)] #plus next state
        else:
          value_function_new[s] += r
    d = max(abs(value_function-value_function_new))
    i+=1
    value_function = value_function_new
  return value_function


def policy_improvement(P, nS, nA, value_from_policy, policy, gamma=0.9):
  """Given the value function from policy improve the policy.
  Parameters
  ----------
  P, nS, nA, gamma:
    defined at beginning of file
  value_from_policy: np.ndarray
    The value calculated from the policy
  policy: np.array
    The previous policy.
  Returns
  -------
  new_policy: np.ndarray[nS]
    An array of integers. Each integer is the optimal action to take
    in that state according to the environment dynamics and the
    given value function.
  """

  new_policy = np.zeros(nS, dtype='int')
 
  for s in range(nS):
    q_a = np.zeros(nA)
    for a in range(nA):
      for p, s_, r, t in P[s][a]:
        if not t:
          q_a[a] += r+gamma*p*value_from_policy[int(s_)]
        else:
          q_a[a]+=r
    new_policy[s] = np.argmax(q_a)
  return new_policy


def policy_iteration(P, nS, nA, gamma=0.9, tol=10e-3):
  """Runs policy iteration.

	You should call the policy_evaluation() and policy_improvement() methods to
	implement this method.

	Parameters
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	tol: float
		tol parameter used in policy_evaluation()
	Returns:
	----------
	value_function: np.ndarray[nS]
	policy: np.ndarray[nS]
  """

  value_function = np.zeros(nS)
  #policy = np.zeros(nS, dtype=int)
  i = 0
  d = 1
  policy = np.random.randint(nA, size=nS)
  while i==0 or d > 0 :
    V = policy_evaluation(P, nS, nA, policy, gamma, tol)
    policy_improve = policy_improvement(P, nS, nA, V, policy, gamma)
    i+=1
    d = np.linalg.norm(policy - policy_improve, ord=1)
    policy = policy_improve 
  return value_function, policy

def value_iteration(P, nS, nA, gamma=0.9, tol=1e-3):
  """
	Learn value function and policy by using value iteration method for a given
	gamma and environment.

	Parameters:
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	tol: float
		Terminate value iteration when
			max |value_function(s) - prev_value_function(s)| < tol
	Returns:
	----------
	value_function: np.ndarray[nS]
	policy: np.ndarray[nS]
  """

  value_function = np.zeros(nS)
  policy = np.zeros(nS, dtype=int)
  d = tol+1
  while d>tol:
    value_function_new = np.zeros(nS)
    for s in range(nS):
      op_a = np.zeros(nA)
      for a in range(nA): 
         for p, s_, r, t in P[s][a]:
           if not t:
             op_a[a] += r+gamma*p*value_function[int(s_)]
           else:
             op_a[a] += r
      value_function_new[s] = np.max(op_a)
      policy[s] = np.argmax(op_a)
    d=max(abs(value_function_new-value_function))
    value_function = value_function_new
  return value_function, policy

def render_single(env, policy, max_steps=100):
  """
    This function does not need to be modified
    Renders policy once on environment. Watch your agent play!

    Parameters
    ----------
    env: gym.core.Environment
      Environment to play on. Must have nS, nA, and P as
      attributes.
    Policy: np.array of shape [env.nS]
      The action to take at a given state
  """

  episode_reward = 0
  ob = env.reset()
  for t in range(max_steps):
    env.render()
    time.sleep(0.25)
    a = policy[ob]
    ob, rew, done, _ = env.step(a)
    episode_reward += rew
    if done:
      break
  env.render();
  if not done:
    print("The agent didn't reach a terminal state in {} steps.".format(max_steps))
  else:
  	print("Episode reward: %f" % episode_reward)